In [6]:
from pycorese.api import CoreseAPI

python_to_java_bridge = 'py4j'
corese = CoreseAPI(java_bridge=python_to_java_bridge)
_ = corese.loadCorese()


import os

if 'src' in os.getcwd():
    os.chdir('..')

os.getcwd()

2025-04-16 21:15:25,662 - INFO - Py4J: Loading CORESE...
2025-04-16 21:15:26,392 - INFO - Py4J: CORESE is loaded


'/mnt/c/repos/rdf_permissions_example'

In [7]:
g = corese.loadRDF("data/server/graph_data.rdf")

In [8]:
# Simple first 5 triples from the whole graph
res = corese.sparqlSelect(g, "SELECT * WHERE { ?s ?p ?o } LIMIT 5", return_dataframe=True)
res

,s,p,o
0,_:b0,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#managedBy
1,_:b1,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#manages...
2,_:b0,http://www.w3.org/1999/02/22-rdf-syntax-ns#rest,_:b1
3,_:b1,http://www.w3.org/1999/02/22-rdf-syntax-ns#rest,http://www.w3.org/1999/02/22-rdf-syntax-ns#nil
4,local://security.permissions.net/entities#4879...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,local://security.permissions.net/vocab#Record


In [9]:
# Since we haven't run ontological reasoning yet, the "manages" relationships are not inferred
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
  FILTER(?p = :manages || ?p = :managesDepartment) .
}

"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,s,p,o
0,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#CHRI...
1,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#BIRT...
2,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#APPAREL
3,local://security.permissions.net/entities#Anya,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#STORAGE
4,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#KITCHEN
5,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOBBY
6,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#OFFICE
7,local://security.permissions.net/entities#Sama...,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOME...


In [10]:
# Run the rules engine
corese.resetRuleEngine(g)
corese.loadRuleEngine(g, profile='owlrl')
corese.loadRuleEngine(g, profile='owlrl_lite')
corese.loadRuleEngine(g, profile='owlrl_ext')
corese.loadRuleEngine(g, profile='rdfs')

JavaObject id=o18

In [12]:
# Now that we have added our inferences, we can see the results of the manages relationships which have been inferred from both managesDepartment and mangedBy
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
  FILTER(?p = :manages || ?p = :managesDepartment) .
}

"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,s,p,o
0,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#CHRI...
1,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#BIRT...
2,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
3,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
4,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
...,...,...,...
61,local://security.permissions.net/entities#Anya,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#STORAGE
62,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#KITCHEN
63,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOBBY
64,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#OFFICE


In [13]:
# Find all the records that a specific user may have access to.

USER = "SalesRep_0"

query = f"""
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select ?record where {{
?department a :Department .
?department :hasItem ?item .
{{
data:{USER} :hasAccess ?country, ?department ; a :User .
?country a :Country .
?record a :Record; :country ?country; :item ?item.}}
union {{
data:{USER} a :Manager .
data:{USER} :managesDepartment ?department .
?record a :Record; :item ?item.
}}
}}
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,record
0,local://security.permissions.net/entities#4436...
1,local://security.permissions.net/entities#4436...
2,local://security.permissions.net/entities#4436...
3,local://security.permissions.net/entities#4436...
4,local://security.permissions.net/entities#4436...
...,...
2110,local://security.permissions.net/entities#4642...
2111,local://security.permissions.net/entities#1783...
2112,local://security.permissions.net/entities#4598...
2113,local://security.permissions.net/entities#1788...


In [15]:
query = """
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select * where {
data:SalesRep_45 a :User ; :hasAccess ?country, ?department .
?department a :Department .
?country a :Country .
?department :hasItem ?item .
?record a :Record; :country ?country; :item ?item.
}
limit 10
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,country,department,item,record
0,local://security.permissions.net/entities#USA,local://security.permissions.net/entities#CHRI...,local://security.permissions.net/entities#477519,local://security.permissions.net/entities#4775...
1,local://security.permissions.net/entities#USA,local://security.permissions.net/entities#CHRI...,local://security.permissions.net/entities#477477,local://security.permissions.net/entities#4774...
2,local://security.permissions.net/entities#USA,local://security.permissions.net/entities#BIRT...,local://security.permissions.net/entities#787395,local://security.permissions.net/entities#7873...
3,local://security.permissions.net/entities#USA,local://security.permissions.net/entities#BIRT...,local://security.permissions.net/entities#451647,local://security.permissions.net/entities#4516...
4,local://security.permissions.net/entities#USA,local://security.permissions.net/entities#CHRI...,local://security.permissions.net/entities#477414,local://security.permissions.net/entities#4774...
5,local://security.permissions.net/entities#USA,local://security.permissions.net/entities#BIRT...,local://security.permissions.net/entities#462567,local://security.permissions.net/entities#4625...
6,local://security.permissions.net/entities#USA,local://security.permissions.net/entities#CHRI...,local://security.permissions.net/entities#490224,local://security.permissions.net/entities#4902...
7,local://security.permissions.net/entities#USA,local://security.permissions.net/entities#CHRI...,local://security.permissions.net/entities#490329,local://security.permissions.net/entities#4903...
8,local://security.permissions.net/entities#USA,local://security.permissions.net/entities#CHRI...,local://security.permissions.net/entities#490371,local://security.permissions.net/entities#4903...
9,local://security.permissions.net/entities#USA,local://security.permissions.net/entities#BIRT...,local://security.permissions.net/entities#999390,local://security.permissions.net/entities#9993...
